In [9]:
import logging

from random import randint
from typing import Optional, Type

import torch
from torch import nn
from torch.nn import init, functional as F

In [52]:
class LinearSubmodule(nn.Module):
    """
    LinearSubmodule composes a Linear layer with an activation and dropout layer
    so that we can have more fine-tuned parameterized control over the layers in
    our MultiLayerPerceptron.

    Parameters
    ----------
        in_dim, units: int, int
            Input/Output dimensions, or number of units, in this particular layer

        dropout: float = 0.5
            Probability that a node will drop out of a given layer, using nn.Dropout

        activation: nn.Module = nn.ReLU
            The activation function applied. (Debating on modifying this to use
            torch.nn.functional.relu instead so we can see the difference between
            learning extra gradients on the ReLU).

    """
    def __init__(
        self,
        in_dim: int,
        units: int,
        dropout: float = 0.5,
        activation: nn.Module = nn.ReLU,
    ):
        super().__init__()
        self.linear = nn.Linear(in_dim, units)
        self.activation = activation()
        self.dropout = nn.Dropout(dropout) if dropout > 0.0 else nn.Identity()

    def forward(self, x):
        x = self.linear(x)
        x = self.activation(x)
        x = self.dropout(x)
        return x


class RepeatedSequential(nn.Sequential):
    """
    RepeatedSequential generates a Sequential submodule using a set of dimensions by
    creating multiple layers of the same moduleclass.

    Parameters
    ----------
        *dims: int
            Input/Output dimensions, or number of units, for each layer. Generates
            a sequence of layer dimensions by zipping.

        moduleclass: Type[nn.Module]
            Factory class used to generate all the layers

        **kwargs
            are forwarded to the moduleclass to configure each layer identically
            execpt for dimensions.

    """
    def __init__(self, *dims: int, moduleclass: Type[nn.Module] = LinearSubmodule, **kwargs):
        super().__init__(*[
            moduleclass(dims[i], dims[i+1], **kwargs)
            for i in range(len(dims) - 1)
        ])


class RandomizedRepeatedSequential(RepeatedSequential):
    """
    RepeatedSequential generates a Sequential submodule using a set of dimensions by
    creating multiple layers of the same moduleclass.

    Parameters
    ----------
        lower: int
            Lower bound on layer dimensions, randomly generated by random.randint

        upper: int
            Upper bound on layer dimensions, randomly generated by random.randint

        layers: int
            number of layers to generate

        **kwargs
            moduleclass and configuration kwargs to configure each layer identically
            execpt for dimensions.

    """
    def __init__(self, lower: int, upper: int, layers: int, **kwargs):
        super().__init__(self, *[randint(lower, upper) for _ in range(layers)], **kwargs)


# TODO:
#     TITLE: Layer dimension generators
#     AUTHOR: frndlytm
#     DESCRIPTION:
#
#         Write a bunch of generators for sequences of dimensions that follow
#         certain growth / decay rules. It could be really interesting to
#         evaluate various MLP dimension arrangements for multiclass classification
#
class MultiLayerPerceptron(nn.Module):
    def __init__(
        self,
        d_in: int,
        d_out: int,
        units: int,
        n_layers: int,
        dropout: float = 0.5,
        shift: Optional[torch.Tensor] = None,
        scale: Optional[torch.Tensor] = None,
    ):
        super().__init__()

        self.shift = nn.Parameter(torch.empty(d_in), requires_grad=False)
        torch.nn.init.zeros_(self.shift)
        if shift is not None:
            self.shift.data = shift

        self.scale = nn.Parameter(torch.empty(d_in), requires_grad=False)
        torch.nn.init.ones_(self.scale)
        if scale is not None:
            self.scale.data = scale

        dims = [d_in, *(units for _ in range(n_layers-1))]
        self.layers = RepeatedSequential(*dims, moduleclass=LinearSubmodule, dropout=dropout)
        self.output = torch.nn.Linear(units, d_out)

    def forward(self, x):
        print(x.shape, self.shift.shape, self.scale.shape)
        x = (x - self.shift) / self.scale
        x = self.layers(x)
        return self.output(x)

    def classify(self, x, threshold: float = 0.5):
        with torch.no_grad():
            # TODO:
            #     TITLE: Strategy for functional layer
            #     AUTHOR: frndlytm
            #     DESCRIPTION:
            #
            #         Decide on how we want to manage final functional layer that
            #         actually performs the classification. This could potentially
            #         be controlled externally.
            #
            y_pred = F.sigmoid(self.forward(x))

        return (y_pred > threshold).float()


In [74]:
import json
import logging
import random
from os import path

import numpy as np
from tqdm import tqdm

from torch import optim
from torch.utils.data import DataLoader, TensorDataset

logging.basicConfig(
    format='%(asctime)s [%(levelname)s] %(name)s - %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S',
    stream=sys.stdout,
)
logging.getLogger('notebook').setLevel(logging.DEBUG)

# Using a seed to maintain consistent and reproducible results
SEED = 100

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [75]:
# If your PC doesn't have enough CPU Ram or Video memory, try decreasing the batch_size
BATCH_SIZE = 128
DATA_DIR = path.join(
    path.dirname(path.dirname(path.realpath("__file__"))),  "data"
)

def load_data(batch_size: int = BATCH_SIZE, datadir: str = DATA_DIR):
    # Grab the feature file and moods targets from listening moods data
    features = np.load(path.join(datadir, 'tp_source_trimmed.npy'), allow_pickle=True)
    moods = np.load(path.join(datadir, f'moods_target_trimmed.npy'), allow_pickle=True)
    moods = np.stack(moods[:, 1]).astype(float)

    # TODO:
    #     TITLE: Uncomment to use shared splits
    #     AUTHOR: frndlytm
    #
    # # Grab the cached indexes from the listening-moods training
    # train_idxs, val_idxs, test_idxs = (
    #     np.load(path.join(datadir, f'train_idx.npy')),
    #     np.load(path.join(datadir, f'val_idx.npy')),
    #     np.load(path.join(datadir, f'test_idx.npy')),
    # )
    size = int(features.shape[1])
    train_size, valid_size, test_size = (
        int(0.6 * size), int(0.2 * size), int(0.2 * size),
    )
    train_idxs, val_idxs, test_idxs = (
        slice(0, train_size, 1),
        slice(train_size, train_size+valid_size, 1),
        slice(train_size+valid_size, -1, 1),
    )

    # Construct TensorDatasets for each index
    train_data, val_data, test_data = (
        TensorDataset(
            torch.from_numpy(features[train_idxs]),
            torch.from_numpy(moods[train_idxs])
        ),
        TensorDataset(
            torch.from_numpy(features[val_idxs]),
            torch.from_numpy(moods[val_idxs])
        ),
        TensorDataset(
            torch.from_numpy(features[test_idxs]),
            torch.from_numpy(moods[test_idxs])
        ),
    )

    # RETURN DataLoader batch iterators on the data
    return {
        "d_in": features.shape[1],
        "d_out": moods.shape[1],
        "datasets":{
            "train": DataLoader(train_data, batch_size=batch_size),
            "valid": DataLoader(val_data, batch_size=batch_size),
            "test": DataLoader(test_data, batch_size=batch_size),
        },
    }


# Load and unpack the data
d_in, d_out, datasets = load_data(batch_size=BATCH_SIZE).values()

In [76]:
# initializing model weights for better convergence
device = "cuda" if torch.cuda.is_available() else "cpu"
model = MultiLayerPerceptron(d_in=d_in, d_out=d_out, units=2, n_layers=2)

# init(model)

optimizer = optim.Adam(model.parameters(), lr=0.001)  # optimizer to train the model
criterion = nn.CrossEntropyLoss()                     # loss criterion

# use gpu if available, These lines move your model to gpu from cpu if available
model = model.to(device)
criterion = criterion.to(device)

# If this line prints cuda, your machine is equipped with a Nvidia GPU and
# PyTorch is utilizing the GPU
print(device)

cpu


In [79]:
EPOCHS = 40

def train(
    model,
    optimizer,
    criterion,
    train_iterator,
    valid_iterator,
    epochs: int = EPOCHS,
    device: str = "cpu"
):

    for epoch in tqdm(range(epochs)):
        train_loss = 0.0 
        valid_loss = 0.0

        # start training
        model.train()
        for (inputs, targets) in train_iterator:
            # zero the gradients from last batch
            # feed the batch to the model
            optimizer.zero_grad()
            inputs.to(device)
            targets.to(device)

            # Evaluate the loss and propagate during training.
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            # Cache the stats for epoch logging
            train_loss += loss.data.item()

        train_loss /= len(train_iterator)

        # start validation
        model.eval()
        with torch.no_grad():
            for (inputs, targets) in valid_iterator:
                # zero the gradients from last batch
                # feed the batch to the model
                optimizer.zero_grad()
                inputs.to(device)
                targets.to(device)

                # Evaluate the loss and propagate during training.
                outputs = model(inputs)
                loss = criterion(outputs, targets)

                valid_loss += loss.data.item()

        valid_loss /= len(valid_iterator)

        # log stats
        print(
            json.dumps(
                {
                    "Epoch": f"{epoch+1:02}",
                    "Training Loss": f"{train_loss:.2f}",
                    "Validation Loss": f"{valid_loss:.2f}",
                }
            )
        )

In [80]:
train(
    model,
    optimizer,
    criterion,
    datasets["train"],
    datasets["valid"],
    epochs=1,
    device=device,
)

100%|██████████| 1/1 [00:00<00:00, 49.97it/s]

torch.Size([120, 200]) torch.Size([200]) torch.Size([200])
torch.Size([40, 200]) torch.Size([200]) torch.Size([200])
{"Epoch": "01", "Training Loss": "69.11", "Validation Loss": "66.56"}


In [ ]:
# testing the accuracy on test set
def test(model, test_iterator):
    test_acc=0

    # Computes without the gradients. Use this while testing your model.
    # As we do not intend to learn from the data
    model.eval()
    with torch.no_grad():
        for (inputs, targets) in test_iterator:
            
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)

    test_acc /= len(test_iterator)

    logging.info(f'Test Acc: {test_acc:.2f}\n')